In [1]:
import os
import json
import re
import nltk
import spacy
import PyPDF2

from openai import OpenAI

In [2]:
os.environ['OPENAI_API_KEY'] = ''

In [10]:
# function that extracts text from a PDF
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        print("Number of PDF pages:", len(reader.pages))
        text = ""
        for  page in reader.pages:
            page_text = page.extract_text()
            text += page_text
            #print(page_text)
    return text

# Chunking

In [3]:
def split_sentences(text):
    
    sentences = re.split('[.|?]', text)
    sentences = [sentence.strip() for sentence in sentences if sentence.split()]

    return sentences

def split_sentences_by_nltk(text):

    chunks = []
    for sentence in nltk.sent_tokenize(text):
        chunks.append(sentence)
    
    return chunks

def split_sentences_by_spacy(text, max_tokens, overlap=0, model='en_core_web_sm'):

    nlp = spacy.load(model)
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]

    return sentences

    tokens_lengths = [count_tokens(sent) for sent in sentences]

    chunks = []
    start_idx = 0

    while start_idx < len(sentences):

        current_chunk = []
        current_token_count = 0
        for idx in range(start_idx, len(sentences)):
            if current_token_count + tokens_lengths[idx] > max_tokens:
                break

            current_chunk.append(sentences[idx])
            current_token_count += tokens_lengths[idx]

        chunks.append(' '.join(current_chunk))

        if overlap >= len(current_chunk):
            start_idx += 1
        else:
            start_idx += len(current_chunk) - overlap


    return chunks

In [4]:
texto = """Entry requirements for Spain are as follows:
              - Passport with validity of at least six months.
              - Return ticket.
           Entry requirements for China are as follows:
              - Passport with validity of at least six months.
              - Return ticket.
              - Visa.
              - Proof of accommodation."""

In [6]:
s_textos = split_sentences_by_nltk(texto)

for i in range(len(s_textos)):
    print(i, s_textos[i])

0 Entry requirements for Spain are as follows:
              - Passport with validity of at least six months.
1 - Return ticket.
2 Entry requirements for China are as follows:
              - Passport with validity of at least six months.
3 - Return ticket.
4 - Visa.
5 - Proof of accommodation.


In [20]:
s_textos = split_sentences_by_spacy(texto, max_tokens=20)
for i in range(0, len(s_textos)):
   print(i, s_textos[i])

0 Entry requirements for Spain are as follows:
              - Passport with validity of at least six months.
              - Return ticket.
           
1 Entry requirements for China are as follows:
              - Passport with validity of at least six months.
              - Return ticket.
              - Visa.
              
2 - Proof of accommodation.


# PDF

In [15]:
texto_pdf = extract_text_from_pdf('/media/armando/Neptuno/git/gen_ai/generative/cpf.pdf')
print(len(texto_pdf))

Number of PDF pages: 348
854806


In [16]:
print(texto_pdf[0:1000])

 
 CÓDIGO PENAL FEDERAL  
 
CÁMARA DE DIPUTADOS DEL H. CONGRESO DE LA UNIÓN 
Secretaría General  
Secretaría de Servicios Parlamentarios  Última s Reforma s DOF 07-06-2024  
 
 
1 de 348 CÓDIGO PENAL FEDERAL  
 
Nuevo Código Publicado en el Diario Oficial de la Federación el 14 de agosto de 1931  
 
TEXTO VIGENTE  
Última s reforma s publicada s DOF 07-06-2024  
 
Disposición derogatoria:  De conformidad con el párrafo primero del artículo  Cuarto Transit orio de la Ley 
Nacional de Ejecución Penal, publicada en el DOF 16 -06-2016 , “se derogan las normas contenidas en el Código 
Penal Federal y leyes especiales de la federación relativas a la remisión parcial de la pena, libertad preparatoria y 
sustitución de la pena durante la ejecución” . 
 
 
 
Al margen un sello que dice: Poder Ejecutivo Federal. - Estados Unidos Mexicanos. -México. - Secretaría 
de Gobernación.  
 
El C. Presidente Constitucional de los Estados Unidos Mexicanos, se ha servido dirigirme el siguiente 
Decreto:  
 